# 1. Hydro: First Data Inspection

## Simulation Overview

In [1]:
using Mera
info = getinfo(400, "../simulations/");

[Mera]: 2019-12-04T19:54:44.517

Code: RAMSES
output [400] summary:
-------------------------------------------------------
simulation time: 594.98 [Myr]
boxlen: 48.0 [kpc]
ncpu: 2048
ndim: 3

amr:           true
level(s): 6 - 14 --> cellsize(s): 750.0 [pc] - 2.93 [pc]

hydro:         true
hydro-variables:  7  --> (:rho, :vx, :vy, :vz, :p, :var6, :var7)
hydro-descriptor: (:density, :velocity_x, :velocity_y, :velocity_z, :thermal_pressure, :passive_scalar_1, :passive_scalar_2)
γ: 1.6667

gravity:       true
particles:     true
- Npart:    5.091500e+05 
- Nstars:   5.066030e+05 
- Ndm:      2.547000e+03 
- Nsinks:   0.000000e+00 

clumps:        true
namelist-file: true

timer-file:       false
compilation-file: true
makefile:         true
patchfile:        true
-------------------------------------------------------



Existing descriptor names for the variables can be used (:density, :velocity_x, :velocity_y, :velocity_z, :thermal_pressure, :passive_scalar_1, :passive_scalar_2) instead of the predefined names (:rho, :vx, :vy, :vz, :p, :var6, :var7). Furthermore, the user has the opportunity to overwrite the full list of descriptor names by changing the entries:

In [4]:
propertynames(info.descriptor)

(:hydro, :usehydro, :hydrofile, :particles, :useparticles, :particlesfile, :gravity, :usegravity, :gravityfile, :clumps, :useclumps, :clumpsfile, :sinks, :usesinks, :sinksfile)

In [5]:
info.descriptor.hydro

7-element Array{Symbol,1}:
 :density         
 :velocity_x      
 :velocity_y      
 :velocity_z      
 :thermal_pressure
 :passive_scalar_1
 :passive_scalar_2

In [6]:
info.descriptor.hydro[2] = :vel_x;

In [7]:
info.descriptor.hydro

7-element Array{Symbol,1}:
 :density         
 :vel_x           
 :velocity_y      
 :velocity_z      
 :thermal_pressure
 :passive_scalar_1
 :passive_scalar_2

These names are now considered in all functions if the following field is set to *true*:

In [8]:
info.descriptor.usehydro = true;

## Load AMR/Hydro Data

In [3]:
info = getinfo(400, "../simulations/", verbose=false); # used to overwrite the previous changes

Read the AMR and the Hydro data from all CPU files of the full box with all existing variables and cell positions (only leaf cells of the AMR grid).

In [4]:
gas = gethydro(info, smallr=1e-30 / info.unit_d);

[Mera]: Get hydro data: 2019-12-04T14:07:08.867

Key vars=(:level, :cx, :cy, :cz)
Using var(s)=(1, 2, 3, 4, 5, 6, 7) = (:rho, :vx, :vy, :vz, :p, :var6, :var7) 

domain:
xmin::xmax: 0.0 :: 1.0  	==> 0.0 [kpc] :: 48.0 [kpc]
ymin::ymax: 0.0 :: 1.0  	==> 0.0 [kpc] :: 48.0 [kpc]
zmin::zmax: 0.0 :: 1.0  	==> 0.0 [kpc] :: 48.0 [kpc]

Reading data...


  5%|██▋                                                |  ETA: 0:03:53

InterruptException: InterruptException:

By default, the predefined hydro-variables are used when all variables are loaded. By changing the flag info.use_hydro_descriptor = true in the InfoType, all functions will automatically use the names assigned to the descriptor list.

The memory consumption of the data table is printed at the end of the function output. We provide a function which gives the possibility to print the used memory of any object: 

In [11]:
usedmemory(gas);

Memory used: 4.52 GB


The assigned data object is now of type *HydroDataType*:

In [6]:
typeof(gas)

HydroDataType

It is a sub-type of to the super-type *DataSetType*

In [10]:
supertype( HydroDataType )

DataSetType

The data is stored in a **JuliaDB** table and the user selected hydro variables and parameters are assigned to fields:

In [13]:
viewfields(gas)


data ==> JuliaDB table: (:columns, :pkey, :perms, :cardinality, :columns_buffer)

info ==> subfields: (:output, :path, :fnames, :simcode, :mtime, :ctime, :ncpu, :ndim, :levelmin, :levelmax, :boxlen, :time, :aexp, :H0, :omega_m, :omega_l, :omega_k, :omega_b, :unit_l, :unit_d, :unit_m, :unit_v, :unit_t, :hydro, :nvarh, :nvarp, :variable_list, :hydro_descriptor, :use_hydro_descriptor, :descriptorfile, :particles_variable_list, :use_particles_descriptor, :particles_descriptor, :particles_descriptorfile, :gravity, :particles, :clumps, :namelist, :headerfile, :makefile, :timerfile, :compilationfile, :Narraysize, :scale, :grid_info, :part_info, :compilation, :constants)

lmin	= 6
lmax	= 14
boxlen	= 48.0
ranges	= [0.0, 1.0, 0.0, 1.0, 0.0, 1.0]
selected_hydrovars	= [-1, 1, 2, 3, 4, 5, 6, 7]
smallr	= 1.4774579400791327e-8
smallc	= 0.0

scale ==> subfields: (:Mpc, :kpc, :pc, :mpc, :ly, :Au, :km, :m, :cm, :mm, :μm, :Msol_pc3, :g_cm3, :Msol_pc2, :g_cm2, :Gyr, :Myr, :yr, :s, :ms, :Msol, :Mearth, :M

For convenience, all the fields from the info-object above (InfoType) are now also accessible from the object with "gas.info".

Print the fields of an object (composite type) in a simple list:

In [15]:
propertynames(gas)

(:data, :info, :lmin, :lmax, :boxlen, :ranges, :selected_hydrovars, :used_descriptors, :smallr, :smallc, :scale)

## Overview of AMR/Hydro

Get an overview of the AMR structure associated with the object *gas* (HydroDataType).
The printed information is stored into the object *amr_overview* as a **JuliaDB** table (code units)  and can be used for further calculations:

In [16]:
overview_amr = amroverview(gas)

Counting...


100%|███████████████████████████████████████████████████| Time: 0:00:35


Table with 9 rows, 4 columns:
level  cells     cellsize    cpus
─────────────────────────────────
6      240956    0.75        52
7      106808    0.375       113
8      327464    0.1875      205
9      991134    0.09375     332
10     2463100   0.046875    562
11     4741974   0.0234375   1086
12     8116352   0.0117188   1832
13     13297129  0.00585938  2046
14     20268216  0.00292969  2024

Get some overview of the data associated with the object *gas*. The calculated information can be accessed from the object *data_overview* (here) in code units for further calculations:

In [3]:
data_overview = dataoverview(gas)

Calculating...


100%|███████████████████████████████████████████████████| Time: 0:16:54


Table with 9 rows, 16 columns:
Columns:
#   colname   type
──────────────────
1   level     Any
2   mass      Any
3   rho_min   Any
4   rho_max   Any
5   vx_min    Any
6   vx_max    Any
7   vy_min    Any
8   vy_max    Any
9   vz_min    Any
10  vz_max    Any
11  p_min     Any
12  p_max     Any
13  var6_min  Any
14  var6_max  Any
15  var7_min  Any
16  var7_max  Any

If the column list is relatively long, the table is typically represented by an overview. To access certain columns, use the *select* function. The representation ":mass" is called a quoted Symbol ([see in Julia documentation](https://docs.julialang.org/en/v1/manual/metaprogramming/#Symbols-1)):

In [9]:
using JuliaDB

In [10]:
select(data_overview, (:level,:mass, :rho_min, :rho_max ) )

Table with 9 rows, 4 columns:
level  mass        rho_min     rho_max
──────────────────────────────────────────
6      1.22391e9   1.47746e-8  0.000431662
7      5.06534e8   1.21721e-5  0.00206836
8      5.06957e8   1.47746e-8  0.009144
9      1.24477e9   1.47746e-8  0.0360798
10     2.39389e9   1.47746e-8  0.131398
11     2.60921e9   1.47746e-8  0.487535
12     2.21069e9   1.47746e-8  1.9644
13     1.84323e9   1.47746e-8  8.01279
14     1.78623e10  1.8429e-5   4.53175e5

Get an array from the column ":mass" in *data_overview* and scale it to the units *Msol*. The order of the calculated data is consistent with the table above:

In [11]:
column(data_overview, :mass) .* info.scale.Msol # '.*" corresponds to an element-wise multiplikation

9-element Array{Float64,1}:
 1.223572065851951e18 
 5.0639435550135654e17
 5.06817168876274e17  
 1.244424770599381e18 
 2.3932318936680315e18
 2.608488691728659e18 
 2.2100749763732588e18
 1.8427186983217672e18
 1.7857323512254865e19

Or simply convert the *:mass* data in the table to *Msol* units by manipulating the column:

In [12]:
data_overview = setcol(data_overview, :mass, :mass => value->value * info.scale.Msol);

┌ Warning: `setcol(t, key::Union{Int, Symbol}, val)` is deprecated, use `transform(t, key => val)` instead.
│   caller = top-level scope at In[12]:1
└ @ Core In[12]:1


In [13]:
select(data_overview, (:level, :mass, :rho_min, :rho_max ) )

Table with 9 rows, 4 columns:
level  mass        rho_min     rho_max
──────────────────────────────────────────
6      1.22357e18  1.47746e-8  0.000431662
7      5.06394e17  1.21721e-5  0.00206836
8      5.06817e17  1.47746e-8  0.009144
9      1.24442e18  1.47746e-8  0.0360798
10     2.39323e18  1.47746e-8  0.131398
11     2.60849e18  1.47746e-8  0.487535
12     2.21007e18  1.47746e-8  1.9644
13     1.84272e18  1.47746e-8  8.01279
14     1.78573e19  1.8429e-5   4.53175e5

## Data Inspection
The data is associated with the field *gas.data* as a **JuliaDB** table (code units). All the loaded properties of each cell are arranged in one row which makes it easy to find, filter, map, aggregate, group the data, etc.
More information can be found in the MERA tutorials or in: [JuliaDB API Reference](http://juliadb.org/latest/api/)

### Table View
Each row corresponds to a cell and its properties to the columns.
The positions cx,cy,cz are the cell-numbers that would correspond to a uniform 3D array for each level. This is used to reconstruct the grid in many functions of **MERA** and should not be modified.

In [14]:
gas.data

Table with 50553133 rows, 12 columns:
Columns:
#   colname  type
────────────────────
1   level    Int64
2   cpu      Int64
3   cx       Int64
4   cy       Int64
5   cz       Int64
6   rho      Float64
7   vx       Float64
8   vy       Float64
9   vz       Float64
10  p        Float64
11  var6     Float64
12  var7     Float64

A more detailed view into the data:

In [15]:
select(gas.data, (:level,:cx, :cy, :cz, :rho) )

Table with 50553133 rows, 5 columns:
level  cx     cy    cz    rho
────────────────────────────────────
6      1      1     1     1.47746e-8
6      1      1     2     1.47746e-8
6      1      1     3     1.47746e-8
6      1      1     4     1.47746e-8
6      1      1     5     1.47746e-8
6      1      1     6     1.47746e-8
6      1      1     7     1.47746e-8
6      1      1     8     1.47746e-8
6      1      1     9     1.47746e-8
6      1      1     10    1.47746e-8
6      1      1     11    1.47746e-8
6      1      1     12    1.47746e-8
⋮
14     13038  8855  8122  1.41561
14     13038  8855  8123  1.29724
14     13038  8855  8124  1.27011
14     13038  8856  8117  1.89495
14     13038  8856  8118  1.83967
14     13038  8856  8119  1.67429
14     13038  8856  8120  1.59913
14     13038  8856  8121  1.45658
14     13038  8856  8122  1.41468
14     13038  8856  8123  1.3037
14     13038  8856  8124  1.29413